# Pricing American Options by COS method
- COS method has no problem finding the conditional expectation as it makes use of the distribution information directly.


As shown in Longstaff and Schwartz’s method {cite:p}`longstaff2001valuing`, we simulate early exercise.

For more fundamentals, see Hull's book {cite:p}`hull2012options`.


- Pricing formulation:

$$ c(x, t_{n-1}) = e^{(-r\Delta t)}\mathbb{E}\left[ V(X_{t_n} \mid X_{t_{n-1}} = x \right] = e^{(-r\Delta t)} \int_{-\infty}^{+\infty} v(y, t_n) f(y \mid x) \, \mathrm{d}y $$
$$ v(x, t_{n-1}) = \max\{ \Lambda(x, t_{n-1}), c(x, t_{n-1}) \} $$

- Replace $c(x, t_{n-1})$ by the COS approximation, i.e.

$$ c(x, t_{n-1}) \approx \hat{c}(x, t_{n-1}) = e^{(-r\Delta t)} {\sum^{+\infty}_{k=0}}' \mathfrak{Re}\left\{ \varphi\left( \frac{k\pi}{b-a} \right)\cdot \exp\left(i k \pi \frac{x -a}{b-a}\right)  \right\}V_k(t_n) $$

where by $'$ we mean that the sum includes the $1/2$-coefficient.  

where:
$$ V_k(t_n) = \begin{cases} 
C_k(a, x^{\star}_n, t_n) + G_k(x^{\star}_n, b) \text{, for a call, } \\
G_k(a, x^{\star}_n) + C_k(a, x^{\star}_n, b) \text{, for a put. } 
\end{cases}$$

$$ G_k(x_1, x_2) = \frac{2}{b-a} \int^{x_2}_{x_1} \Lambda(x, t_n)\cos\left(k\pi \frac{x-a}{b-a}\right) \, \mathrm{d}x$$

$$ C_k(x_1, x_2, t_n) = \frac{2}{b-a} \int^{x_2}_{x_1} c(x, t_n)\cos\left(k\pi \frac{x-a}{b-a}\right) \, \mathrm{d}x$$

- Further replacing $c(x, t_n)$ by $\hat{c}(x, t_{n-1})$ in the defintion of $V_k(t_n)$, yields $\hat{V}_k(t_n)$.

$$ \hat{V}_k(t_n) =  $$

- Let us implement this. Computational complexity - $NK\log_2{K}$


In [1]:
# Import libraries
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy as ss
from scipy.stats import norm
from scipy.integrate import trapezoid
import pandas as pd

In [2]:
# Recovery CDF
def cos_cdf(a, b, omega, chf, x):
    # F_k coefficients
    F_k = 2.0 / (b - a) * np.real(chf * np.exp(-1j * omega * a))
    cdf = np.squeeze(F_k[0] / 2.0 * (x - a)) + np.matmul(F_k[1:] / omega[1:], np.sin(np.outer(omega[1:], x - a)))
    return cdf

# Recover PDF
def cos_pdf(a, b, omega, chf, x): 
    # F_k coefficients
    F_k = 2.0 / (b - a) * np.real(chf * np.exp(-1j * omega * a))
    # Separate first term and remaining terms like in cos_cdf
    pdf = F_k[0] / 2.0 + np.matmul(F_k[1:], np.cos(np.outer(omega[1:], x - a)))
    return pdf

# exact price - Black-Scholes formula; from ftcs_bs_put
def bs_put_price(S0, K, r, sigma, T):
    d1 = ( np.log(S0/K) + (r + 0.5*sigma**2) * T ) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    bs_put = -1 * norm.cdf(-d1)*S0 + norm.cdf(-d2)*K*np.exp(-r*T)
    return bs_put

# MC paths generator with antithetic sampling
def simulate_gbm_paths(S0, r, sigma, T, n_steps, n_paths):
    """ Antithetic: We input 50,000 paths and we get 100,000 paths """
    np.random.seed(1) 
    dt = T / n_steps

    # Generate randoms for half of n_paths
    Z = np.random.randn(n_paths, n_steps)
    
    # Create vector of paths and antithetic paths
    Z_all = np.vstack([Z, -Z])  
    
    # Initialize paths
    n_total = 2 * n_paths
    S = np.zeros((n_total, n_steps + 1))
    S[:, 0] = S0
    
    # Simulate paths
    for t in range(n_steps):
        S[:, t+1] = S[:, t] * np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z_all[:, t])
    
    return S